In [1]:
import pandas as pd
import sys
import constants
from sklearn.model_selection import train_test_split
import numpy as np
import os

Read file

In [2]:
from preprocessing import transform_and_import_data
listings = transform_and_import_data(constants.DATA_PATH)

In [8]:
listings.set_index("id", inplace=True)
listings.drop(['name', 'description', "neighborhood_overview"], axis=1, inplace=True)

In [9]:
listings.head()

,host_response_time,host_is_superhost,host_listings_count,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable
id,,,,,,,,,,,,,,,,,,,,,
10595,within an hour,t,6.0,6.0,t,ΑΜΠΕΛΟΚΗΠΟΙ,37.98863,23.76527,Entire rental unit,Entire home/apt,...,33,4.78,4.82,4.76,4.85,4.85,4.52,4.67,t,t
10990,within an hour,t,6.0,6.0,t,ΑΜΠΕΛΟΚΗΠΟΙ,37.98903,23.76448,Entire rental unit,Entire home/apt,...,62,4.80,4.84,4.84,4.89,4.89,4.77,4.74,t,t
10993,within an hour,t,6.0,6.0,t,ΑΜΠΕΛΟΚΗΠΟΙ,37.98888,23.76473,Entire rental unit,Entire home/apt,...,87,4.83,4.91,4.93,4.98,4.98,4.84,4.78,t,t
2219385,within an hour,f,2.0,2.0,t,ΣΤΑΔΙΟ,37.96538,23.74292,Entire condo,Entire home/apt,...,29,4.62,4.57,4.41,4.75,4.79,4.61,4.64,t,t
10995,within an hour,t,6.0,6.0,t,ΑΜΠΕΛΟΚΗΠΟΙ,37.98903,23.76448,Entire rental unit,Entire home/apt,...,29,4.83,4.96,4.93,4.93,4.89,4.81,4.81,t,t


Transform categorical, etc.

In [ ]:
for column in data.columns:
    
    if data[column].dtype == np.number:
        continue

# TODO: AM: vs OneHotEncoder
data[column] = LabelEncoder().fit_transform(data[column])

Split X, Y and train/test

In [9]:
#separating the data and label
X = data.drop(columns='price', axis=1).values
y = data['price'].values

# split the data into train and test set
# Split the data into training and test set
X_train_o, X_test, y_train_o, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

# Split the data into training and Validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_o, y_train_o, test_size=0.15, random_state=24)

print(len(X_train))
print(len(X_test))

7184
3079


```
id
name
host_id
host_name
host_is_superhost
host_identity_verified
neighbourhood_cleansed
latitude
longitude
city_name
room_type
accommodates
bathrooms_text
bedrooms
beds
number_of_reviews
review_scores_rating
review_scores_accuracy
review_scores_cleanliness
review_scores_checkin
review_scores_communication
review_scores_location
review_scores_value
license
instant_bookable
nearest_stop_m
n_pois_3klm_radius
```

Scaler

- MinMax
- Robust

In [ ]:
# Initialise the Scaler
min_max_scaler = MinMaxScaler()

# Fit and transform train data and then transform the test data based on the transformed train.
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [ ]:
data.info()

In [ ]:
from keras.model import Model, Sequential

In [ ]:
# Determine the input size of the algorithm
input_shape = len(X_train.columns)

model = Sequential()

model.add(Dense(128, activation='relu', input_shape=input_shape))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# softmax classification
model.add(Dense(num_classes, activation='softmax'))

model.summary()

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
# Callbacks
es = EarlyStopping(monitor='val_accuracy', patience=8, min_delta=1)
filepath="Skin_lesion_characterization/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint] #PlotLossesKeras()

In [ ]:
batch_size = BATCH_SIZE
epochs = 15

history_custom = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size = batch_size,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=[PlotLossesKeras()])

# save model

score = model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

Evalution

model.evaluate



Adjust Model

In [ ]:
plt.subplot(1,2,1)
plt.plot(image_mobilenet.history['accuracy'], marker='o')
plt.plot(image_mobilenet.history['val_accuracy'], marker='o')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')

plt.subplot(1,2,2)
plt.plot(image_mobilenet.history['loss'], marker='o')
plt.plot(image_mobilenet.history['val_loss'], marker='o')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()